This file contains the necessary code to get the counts and metrics for each company and then runs it through the various ML models

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression
import numpy as np
import csv
import os
import pandas as pd
import math

In [2]:
dir = 'C:/Users/Chaitu Konjeti/socweb-glassdoor-project/REVIEWS/'

In [3]:
all_metrics = {}

metrics_file = pd.read_csv('metrics.csv')

# get all the proper metrics from the metrics file
for row in metrics_file.index:
    #print(int(metrics_file['Data Year - Fiscal'][row]))
    if metrics_file['Company Name'][row] not in all_metrics.keys():
        all_metrics[metrics_file['Company Name'][row]] = []
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])) and not int(metrics_file['Data Year - Fiscal'][row]) != '0':
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))
    else:
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])):
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
metrics = []
list_of_posts = []

#iterate through all files in the directory
for filename in os.listdir(dir):
    print(filename)
    df = pd.read_csv(dir + filename, header=0)

    #get dates and pros columns from data file
    dates = df['date']
    pros = df['pros']

    #get the list of years for all the posts in the file and create a dictionary with years as the keys
    years = [date.split(' ')[3] for date in dates]
    corpus = {key: '' for key in set(years)}

    #add the post to the proper year in the dictionary
    for date, pro in zip(years, pros):
        corpus[date] += pro + ' '

    #sort the keys in order
    keys = sorted(corpus)
    
    company_name = filename.split('.')[0]
    
    metric_years = [val[1] for val in set(all_metrics[company_name]) if val[1] in years]
    
    metrics.extend([val[0] for val in set(all_metrics[company_name]) if val[1] in years])
    
    #only use the metrics from years that present in the datafile
    for key in keys:
        if key in metric_years:
            list_of_posts.append(corpus[key])
    
#creates count vectorizer for posts
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 3))
counts = vectorizer.fit_transform(list_of_posts)

metrics = np.asarray(metrics)

AMERICAN AIRLINES GROUP INC.csv
APPLE INC.csv
AVNET INC.csv


In [7]:
print(counts.shape)

(28, 8511)


In [9]:
print(metrics.shape)

(28,)


In [10]:
clf = RandomForestClassifier(max_depth=5)
clf.fit(counts, metrics)

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
pred = []
for i in range(counts.shape[1]):
    if i % 2 == 0:
        pred.append(1)
    else:
        pred.append(1)
print(clf.predict([pred]))

[2524]
